In [1]:
import pandas as pd

In [2]:
from utils import get_dir_path
from pathlib import Path
dir_path = get_dir_path()

In [3]:
wacc = pd.read_csv(Path(dir_path,'csv/My Wacc Report.csv'))
shares = pd.read_csv(Path(dir_path,'csv/My Shares Values.csv'))

In [4]:
merged_df = pd.merge(wacc, shares, left_on='Scrip Name', right_on='Scrip')
result = merged_df[['Scrip Name', 'WACC Rate']]

In [5]:
result.to_csv(Path(dir_path,'csv/My Wacc Rate.csv'), index=False)

In [9]:
from database import Tracker, User, get_db, Scripts

async with get_db() as db:
    users = db.execute(User).all()
    for index, row in result.iterrows():
        script = db.execute(Scripts).filter(Scripts.ticker == row['Scrip Name']).first()
        if not script:
            continue
        for user in users:
            tracker = Tracker(script=script, user=user, price=row['WACC Rate'], price_delta=10)
            db.add(tracker)
            db.commit()